In [157]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.
Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format

Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):

In [158]:
prod_activations_logs = pd.read_csv('/home/jupyter-l.fedoseeva-12/Lesson_5/Dop_zadanie/prod_activations_logs.csv', sep=';')

In [159]:
tm_sales_1 = pd.read_csv('/home/jupyter-l.fedoseeva-12/Lesson_5/Dop_zadanie/tm_sales_1.csv', sep=';')
tm_sales_2 = pd.read_csv('/home/jupyter-l.fedoseeva-12/Lesson_5/Dop_zadanie/tm_sales_2.csv', sep=';')
tm_sales_3 = pd.read_csv('/home/jupyter-l.fedoseeva-12/Lesson_5/Dop_zadanie/tm_sales_3.csv', sep=';')

In [160]:
tm_sales_1.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [161]:
tm_sales_2

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [162]:
prod_activations_logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
dtype: object

In [163]:
tm_sales_2

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [164]:
tm_sales_3

,FILIAL_ID,PROD_ID,SUBS_ID,ACT_DTTM
0,3,1499,id5764122,18-03-2020 15:44
1,6,3020,id7642700,15-03-2020 14:21
2,2,5677,id1374509,17-03-2020 11:48


In [165]:
tm_sales_1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25


In [166]:
all_tm_sales_1 = pd.concat([tm_sales_1, tm_sales_2, tm_sales_3])

In [167]:
all_tm_sales_1 = all_tm_sales_1.reset_index(drop=True)

In [168]:
all_tm_sales_1 = all_tm_sales_1.dropna()

In [169]:
all_tm_sales_1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [170]:
def result(cell):
    if cell.startswith('id'):
        return cell
    return 'id{}'.format(cell)

In [171]:
all_tm_sales_1['SUBS_ID'] = all_tm_sales_1.SUBS_ID.apply(result)

In [172]:
all_tm_sales_1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42
6,id2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [173]:
all_tm_sales_1 = all_tm_sales_1.merge(prod_activations_logs, how='inner', on=['SUBS_ID', 'PROD_ID'])

In [174]:
all_tm_sales_1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,5,6431,19-03-2020 13:00,19-03-2020 13:00,19-03-2020 13:03
2,id5416547,3,1743,17-03-2020 10:17,17-03-2020 10:17,25-03-2020 11:00
3,id2185490,2,3210,16-03-2020 16:28,16-03-2020 16:28,01-12-2020 00:00
4,id7642700,6,3020,15-03-2020 14:21,15-03-2020 14:21,15-03-2020 23:42


In [175]:
all_tm_sales_1.START_DTTM = pd.to_datetime(all_tm_sales_1['START_DTTM'], format='%d-%m-%Y %H:%M')
all_tm_sales_1.END_DTTM = pd.to_datetime(all_tm_sales_1['END_DTTM'], format='%d-%m-%Y %H:%M')

In [180]:
all_tm_sales_1['raznitsa'] = all_tm_sales_1.END_DTTM - all_tm_sales_1.START_DTTM

In [181]:
all_tm_sales_1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,raznitsa
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [182]:
greater_than_5min = all_tm_sales_1['raznitsa'] > '5m'

In [183]:
greater_than_5min

0     True
1    False
2     True
3     True
4     True
Name: raznitsa, dtype: bool

In [185]:
final_data = all_tm_sales_1.loc[greater_than_5min]

In [200]:
final_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,raznitsa
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [191]:
final_data.SUBS_ID.sort_values().str.cat(sep=', ')

'id2185490, id4651830, id5416547, id7642700'

In [204]:
final_data[['FILIAL_ID', 'ACT_DTTM', 'raznitsa']]

,FILIAL_ID,ACT_DTTM,raznitsa
0,1,20-03-2020 14:59,255 days 09:01:00
2,3,17-03-2020 10:17,8 days 00:43:00
3,2,16-03-2020 16:28,259 days 07:32:00
4,6,15-03-2020 14:21,0 days 09:21:00


In [205]:
final_data[['ACT_DTTM', 'FILIAL_ID', 'raznitsa']] \
                   .to_csv('final_data_correction.csv', sep=';', index=False)